In [1]:
import pandas as pd
import json
import os

In [3]:
# evo related libraries and funcs
from evo.core import metrics
from evo.tools import log
log.configure_logging(verbose=True, debug=True, silent=False)

import pprint
import numpy as np

from evo.tools import plot
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook
from evo.tools import file_interface
from evo.core import sync
import copy
# temporarily override some package settings
from evo.tools.settings import SETTINGS
SETTINGS.plot_usetex = False

from mathutils import Matrix, Vector
def get_tq_from_matrix(RT):
    t = RT.decompose()[0]
    q = RT.decompose()[1]
    tq = [t[0], t[1], t[2], q.x, q.y, q.z, q.w]
    return tq
def get_RcC_from_RT(RT):
    R_cv2bcam = Matrix(
        ((1, 0,  0),
        (0, -1, 0),
        (0, 0, -1)))
    R = Matrix((RT[0][0:3],
                RT[1][0:3],
                RT[2][0:3])) # direction of world axes in cam frame
    t = Vector((RT[0][3],
                RT[1][3],
                RT[2][3])) # pos of world origin in cam frame
    # Note: axes position direction correction from cv space to world space
    R = R_cv2bcam @ R
    t = R_cv2bcam @ t
    Rc = R.transposed()    # camera rotation in world coord
    C = -1 * Rc @ t        # pos of cam center in world coord

    RcC = Matrix((
                Rc[0][:] + (C[0],),
                Rc[1][:] + (C[1],),
                Rc[2][:] + (C[2],),
                (0, 0, 0, 1)))
    return RcC
def get_reconstructed_traj_file(source, dest):
    file = open(source, "r")
    num = int(file.readline()[0:-1])
    cam_extmat_tq_rec = []
    for fr_num in range(num):
        ext_mat = np.identity(4)
        row1 = file.readline()[0:-1].split(' ')

        ext_mat[0,:] = [float(_) for _ in row1[1:]]
        ext_mat[1,:] = [float(_) for _ in file.readline()[0:-1].split(' ')]
        ext_mat[2,:] = [float(_) for _ in file.readline()[0:-1].split(' ')]
        ext_mat = Matrix(ext_mat)
        tq = get_tq_from_matrix(get_RcC_from_RT(ext_mat))
#         tq[0] = -tq[0]
        tq.insert(0, int(row1[0]))
        cam_extmat_tq_rec.append(tq)
    cam_extmat_tq_rec = np.matrix(cam_extmat_tq_rec)
    np.savetxt(dest, cam_extmat_tq_rec, fmt='%.5f')
    
def calc_pe(pose_relation, traj_ref, traj_est_aligned):
    """APE""" 
    data = (traj_ref, traj_est_aligned) 
    ape_metric = metrics.APE(pose_relation)
    ape_metric.process_data(data)
    ape_mean, ape_std = ape_metric.get_statistic(metrics.StatisticsType.mean), ape_metric.get_statistic(metrics.StatisticsType.std)

    """RPE"""
    # normal mode
    delta = 1
    delta_unit = metrics.Unit.frames
    # all pairs mode
    all_pairs = False 
    data = (traj_ref, traj_est_aligned)
    rpe_metric = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
    rpe_metric.process_data(data)
    rpe_mean, rpe_std = rpe_metric.get_statistic(metrics.StatisticsType.mean), rpe_metric.get_statistic(metrics.StatisticsType.std)

    return ape_mean, rpe_mean
def get_evo_analysis(exp_dir, ind_largest, show_plot=False):
    # get recon_traj_file
    get_reconstructed_traj_file(
        source=exp_dir+'/base_experiment/sfm/results/model-'+str(ind_largest)+'-cams.txt', 
        dest=exp_dir+'/base_experiment/sfm/results/model-'+str(ind_largest)+'-cams_wmat_tq.txt')

    # evo analysis
    ref_file = exp_dir + "/base_data/cam_wmat_tq_gt.txt"
    est_file = exp_dir + '/base_experiment/sfm/results/model-'+str(ind_largest)+'-cams_wmat_tq.txt'
    traj_ref = file_interface.read_tum_trajectory_file(ref_file)
    traj_est = file_interface.read_tum_trajectory_file(est_file)   

    max_diff = 0.1
    traj_ref, traj_est = sync.associate_trajectories(traj_ref, traj_est, max_diff)

    traj_est_aligned = copy.deepcopy(traj_est)
    traj_est_aligned.align(traj_ref, correct_scale=True, correct_only_scale=False)

    if show_plot:
        fig = plt.figure()
        traj_by_label = {
        #     "estimate (not aligned)": traj_est,
            "reconstruction (aligned)": traj_est_aligned,
            "groundtruth": traj_ref
        }
        plot.trajectories(fig, traj_by_label, plot.PlotMode.xyz)
        ax = plt.gca(); ax.set_facecolor((1,1,1))
        plt.axis('off')
        plt.show()

    ape_RT, rpe_RT = calc_pe(metrics.PoseRelation.full_transformation, traj_ref, traj_est_aligned)
    ape_T, rpe_T = calc_pe(metrics.PoseRelation.translation_part, traj_ref, traj_est_aligned)
    ape_R, rpe_R = calc_pe(metrics.PoseRelation.rotation_part, traj_ref, traj_est_aligned)
    return ape_RT, rpe_RT, ape_T, rpe_T, ape_R, rpe_R

[DEBUG][2021-01-02 22:40:13,884][log.configure_logging():115]
System info:
Python 3.7.4
Linux-4.15.0-128-generic-x86_64-with-debian-stretch-sid
hpl@titan-ubuntu



In [4]:
csv_fp = '/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/experiment_summary.csv'
summ = pd.read_csv(csv_fp, header=0, index_col=0, sep=',', keep_default_na=False)
summ

,cysto_len,features,fraction,L2_reproj_err,ind_largest,mesh_vertices,mesh_faces,n_sfm_views,n_tex_views,tex_faces,...,ape_T,rpe_T,ape_R,rpe_R,cc_rms,cc_overlap,mv_eval_comp,mv_eval_acc,tex_acc,Unnamed: 24
EXP,,,,,,,,,,,,,,,,,,,,,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsis_t6_s1,4,1522,1,0.62,0,22160,44316,120,120,44311,...,0.00009,0.00529,2.82842,0.00341,,,,,x,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsis_t8_s1,5.3,1531,1,0.63,0,27695,55380,160,160,55357,...,0.00008,0.00008,0.01026,0.00327,,,,,a,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsis_t10_s1,6.7,1529,1,0.63,0,32337,64670,200,200,64642,...,0.00008,0.00008,0.00991,0.00342,,,,,a,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsis_t12_s1,8,1530,1,0.63,0,36249,72494,240,240,72356,...,0.00008,0.00523,2.82842,0.00384,,,,,x,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsis_t14_s1,9.3,1525,1,0.63,0,41533,83062,280,280,83014,...,0.00008,0.00523,2.82842,0.00354,,,,,x,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsps_t6_s1,8,1472,1,0.62,0,44166,88327,240,240,88273,...,0.00007,0.00131,2.82842,0.00219,,,,,x,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsps_t8_s1,10.7,1469,1,0.63,0,51967,103930,320,320,103892,...,0.00007,0.00006,0.00909,0.00231,,,,,a,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsps_t10_s1,13.3,1475,1,0.63,0,56464,112924,400,400,112732,...,0.00007,0.00006,0.00991,0.00342,0.000422,1,1 1 1 1 1 1,1 1 1 1 1 1,a,
/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/Ms-Tsps_t12_s1,16,1472,1,0.63,0,61260,122515,480,480,122484,...,0.00007,0.00006,0.00998,0.00276,,,,,a,


In [11]:
exs = summ['cysto_len'].index.tolist()
for ex_dir in exs:
    if not isinstance(ex_dir, str) or ex_dir == '':
        continue # ignore empty line and go to next loop
    if not os.path.exists(ex_dir):
        print('This experiment folder doesn\'t exist!')
        break # exit code and check summary csv file
    if summ['cysto_len'][ex_dir] != '':
        print(ex_dir, " is already evaluated")
        continue # ignore filled line

    # read recon_results.json
    file = open(ex_dir+'/base_experiment/recon_results.json', "r")
    recon_result = json.load(file)
    file.close()
    summ['cysto_len'][ex_dir] = str(round(float(recon_result["cysto_len"]),1))
    summ['features'][ex_dir] = str(round(float(recon_result["features_avg"])))
    summ['fraction'][ex_dir] = str(round(float(recon_result["fraction_images_reconstructed_largest"]), 2))
    summ['L2_reproj_err'][ex_dir] = str(round(float(recon_result["l2_reprojection_error_largest"]), 2))
    ind_largest = recon_result["l2_reprojection_error_all"].index(recon_result["l2_reprojection_error_largest"])
    summ['ind_largest'][ex_dir] = str(ind_largest)
    summ['mesh_vertices'][ex_dir] = str(round(float(recon_result["mesh_vertices"])))
    summ['mesh_faces'][ex_dir] = str(round(float(recon_result["mesh_faces"])))
    summ['tex_faces'][ex_dir] = str(round(float(recon_result["tex_faces"])))
    summ['tex_faces_pct'][ex_dir] = str(round(float(recon_result["pct_textured_faces"]), 2))
    summ['n_sfm_views'][ex_dir] = str(round(float(recon_result["n_sfm_images"])))
    summ['n_tex_views'][ex_dir] = str(round(float(recon_result["n_tex_images"])))
    # run evo analysis
    ape_RT, rpe_RT, ape_T, rpe_T, ape_R, rpe_R = get_evo_analysis(ex_dir, ind_largest)
    summ['ape_RT'][ex_dir] = str(round(ape_RT, 5))
    summ['rpe_RT'][ex_dir] = str(round(rpe_RT, 5))
    summ['ape_T'][ex_dir] = str(round(ape_T, 5))
    summ['rpe_T'][ex_dir] = str(round(rpe_T, 5))
    summ['ape_R'][ex_dir] = str(round(ape_R, 5))
    summ['rpe_R'][ex_dir] = str(round(rpe_R, 5))  
summ.to_csv('/home/hpl/Documents/cysto3D/EndoVidSynthesis/data/experiment_summary_updated.csv')